# Day 12: Random Forests - Bagging, OOB Intuition, and Feature Importance

**MGMT 47400 - Predictive Analytics**  
**4-Week Online Course**  
**Day 12 - Wednesday June 2, 2027**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/davi-moreira/2026Summer_predictive_analytics_purdue_MGMT474/blob/main/notebooks/12_random_forests_importance.ipynb)

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Explain bagging and why forests reduce variance
2. Train a random forest and tune the most impactful knobs
3. Use permutation importance responsibly
4. Compare forest vs tree vs linear/logistic baselines
5. Produce project-ready model comparison tables

---

In [ ]:
# Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.inspection import permutation_importance
from sklearn.metrics import roc_auc_score
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.precision', 4)
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
print("✓ Setup complete!")

## 1. From Single Tree to Forest: The Bagging Idea

### The Variance Problem with Single Trees

**Problem:** Decision trees are unstable
- Small change in data → completely different tree
- High variance in predictions
- Overfitting on individual quirks

### The Solution: Bootstrap Aggregating (Bagging)

**Algorithm:**
1. Create B bootstrap samples (random sampling with replacement)
2. Train one tree on each bootstrap sample
3. Aggregate predictions (average for regression, vote for classification)

**Why it works:**
- Averaging reduces variance
- Each tree sees slightly different data
- Errors cancel out through averaging

### Random Forest = Bagging + Random Feature Selection

**Extra randomness:** At each split, only consider random subset of features
- Decorrelates trees
- Prevents one strong feature from dominating
- Further variance reduction

In [ ]:
# Load data
data = load_breast_cancer(as_frame=True)
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_SEED, stratify=y)

print(f"Train: {len(X_train)} | Test: {len(X_test)}")
print(f"Features: {X.shape[1]}")

## 2. Single Tree vs Random Forest

In [ ]:
# Compare single tree vs forest
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)

# Single tree (tuned depth)
tree = DecisionTreeClassifier(max_depth=5, random_state=RANDOM_SEED)
tree_scores = cross_val_score(tree, X_train, y_train, cv=cv, scoring='roc_auc')

# Random forest
forest = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=RANDOM_SEED)
forest_scores = cross_val_score(forest, X_train, y_train, cv=cv, scoring='roc_auc')

print("=== SINGLE TREE VS RANDOM FOREST ===")
print(f"\nSingle Tree (depth=5):")
print(f"  CV ROC-AUC: {tree_scores.mean():.4f} ± {tree_scores.std():.4f}")
print(f"  Fold scores: {tree_scores.round(4)}")

print(f"\nRandom Forest (100 trees, depth=5):")
print(f"  CV ROC-AUC: {forest_scores.mean():.4f} ± {forest_scores.std():.4f}")
print(f"  Fold scores: {forest_scores.round(4)}")

print(f"\n=== IMPROVEMENT ===")
print(f"Mean improvement: {(forest_scores.mean() - tree_scores.mean()):.4f}")
print(f"Variance reduction: {(tree_scores.std() - forest_scores.std()):.4f}")

# Visualize
fig, ax = plt.subplots(figsize=(10, 6))
positions = [1, 2]
bp = ax.boxplot([tree_scores, forest_scores], positions=positions, widths=0.6, patch_artist=True)
for patch, color in zip(bp['boxes'], ['lightblue', 'lightgreen']):
    patch.set_facecolor(color)
ax.set_xticklabels(['Single Tree', 'Random Forest'])
ax.set_ylabel('ROC-AUC Score')
ax.set_title('Variance Reduction: Single Tree vs Random Forest')
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print("\n💡 Forest has higher mean AND lower variance")
print("💡 More stable predictions across different data samples")

## 3. Tuning Random Forests

### Most Important Hyperparameters

**n_estimators** (number of trees)
- More trees = better performance (usually)
- Diminishing returns after ~100-500
- More trees = longer training time

**max_features** (features per split)
- sqrt(n_features) for classification (default)
- n_features/3 for regression
- Lower values = more decorrelation

**max_depth** (tree depth)
- Controls individual tree complexity
- None = grow until pure (common for forests)

**min_samples_split** (minimum samples to split)
- Higher values = simpler trees
- Prevents overfitting

In [ ]:
# Effect of number of trees
n_estimators_range = [10, 25, 50, 100, 200, 300]
results = []

for n_est in n_estimators_range:
    rf = RandomForestClassifier(n_estimators=n_est, random_state=RANDOM_SEED, n_jobs=-1)
    scores = cross_val_score(rf, X_train, y_train, cv=cv, scoring='roc_auc')
    results.append({
        'n_estimators': n_est,
        'cv_mean': scores.mean(),
        'cv_std': scores.std()
    })

results_df = pd.DataFrame(results)
print("=== N_ESTIMATORS SWEEP ===")
print(results_df.to_string(index=False))

# Plot
plt.figure(figsize=(10, 6))
plt.errorbar(results_df['n_estimators'], results_df['cv_mean'], 
             yerr=results_df['cv_std'], marker='o', capsize=5, linewidth=2)
plt.xlabel('Number of Trees')
plt.ylabel('CV ROC-AUC')
plt.title('Random Forest: Effect of Number of Trees')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n💡 Performance plateaus after ~100-200 trees")
print("💡 Use more trees for final model, fewer for experimentation")

## 📝 PAUSE-AND-DO Exercise 1 (10 minutes)

**Task:** Tune `n_estimators` and `max_features` minimally and report effects.

---

In [ ]:
# YOUR CODE: Tune max_features
import math

max_features_options = ['sqrt', 'log2', 0.3, 0.5, None]
tuning_results = []

for max_feat in max_features_options:
    rf = RandomForestClassifier(
        n_estimators=100,
        max_features=max_feat,
        random_state=RANDOM_SEED,
        n_jobs=-1
    )
    scores = cross_val_score(rf, X_train, y_train, cv=cv, scoring='roc_auc')
    tuning_results.append({
        'max_features': str(max_feat),
        'cv_mean': scores.mean(),
        'cv_std': scores.std()
    })

tuning_df = pd.DataFrame(tuning_results)
print("=== MAX_FEATURES TUNING ===")
print(tuning_df.to_string(index=False))

best_idx = tuning_df['cv_mean'].idxmax()
print(f"\n✓ Best max_features: {tuning_df.loc[best_idx, 'max_features']} ({tuning_df.loc[best_idx, 'cv_mean']:.4f})")

### YOUR ANALYSIS:

**Effect of max_features:**  
[What did you observe? How does it affect performance?]

**Recommendation:**  
[Which value would you use in production? Why?]

---

## 4. Out-of-Bag (OOB) Score

### Free Cross-Validation

**Insight:** Each tree is trained on ~63% of data (bootstrap)
- Remaining ~37% is "out-of-bag" for that tree
- Can use OOB samples to estimate test performance
- No need for separate validation set!

**OOB Score ≈ Cross-Validation Score**
- Faster than CV
- Uses all data for training
- Good for initial model selection

In [ ]:
# Compare OOB vs CV
rf_oob = RandomForestClassifier(
    n_estimators=100,
    oob_score=True,
    random_state=RANDOM_SEED,
    n_jobs=-1
)

rf_oob.fit(X_train, y_train)
oob_score = rf_oob.oob_score_

# Compare to CV
cv_scores = cross_val_score(rf_oob, X_train, y_train, cv=cv, scoring='accuracy')

print("=== OOB VS CROSS-VALIDATION ===")
print(f"OOB Score (free): {oob_score:.4f}")
print(f"CV Score (5-fold): {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
print(f"Difference: {abs(oob_score - cv_scores.mean()):.4f}")

print("\n💡 OOB score is a good proxy for test performance")
print("💡 Use OOB for quick iterations, CV for final evaluation")

## 5. Feature Importance

### Two Types of Importance

**1. Gini/Entropy Importance (built-in)**
- Based on how much each feature reduces impurity
- Fast to compute
- ⚠️ Biased toward high-cardinality features
- ⚠️ Can be misleading with correlated features

**2. Permutation Importance (recommended)**
- Shuffle feature, measure performance drop
- More reliable
- Slower to compute
- Works for any model

In [ ]:
# Train final forest
rf_final = RandomForestClassifier(n_estimators=200, random_state=RANDOM_SEED, n_jobs=-1)
rf_final.fit(X_train, y_train)

# Built-in importance
builtin_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_final.feature_importances_
}).sort_values('importance', ascending=False)

print("=== BUILT-IN FEATURE IMPORTANCE (Top 10) ===")
print(builtin_importance.head(10).to_string(index=False))

# Visualize
plt.figure(figsize=(10, 8))
top_features = builtin_importance.head(15)
plt.barh(range(len(top_features)), top_features['importance'])
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Gini Importance')
plt.title('Top 15 Features by Built-in Importance')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 6. Permutation Importance (Recommended)

In [ ]:
# Permutation importance
perm_importance = permutation_importance(
    rf_final, X_test, y_test,
    n_repeats=10,
    random_state=RANDOM_SEED,
    scoring='roc_auc'
)

perm_df = pd.DataFrame({
    'feature': X.columns,
    'importance_mean': perm_importance.importances_mean,
    'importance_std': perm_importance.importances_std
}).sort_values('importance_mean', ascending=False)

print("=== PERMUTATION IMPORTANCE (Top 10) ===")
print(perm_df.head(10).to_string(index=False))

# Compare built-in vs permutation
comparison = builtin_importance.merge(
    perm_df,
    on='feature',
    suffixes=('_builtin', '_perm')
).head(10)

print("\n=== TOP 10: BUILT-IN VS PERMUTATION ===")
print(comparison[['feature', 'importance', 'importance_mean']].to_string(index=False))

## 📝 PAUSE-AND-DO Exercise 2 (10 minutes)

**Task:** Compute permutation importance and write 3 interpretation bullets.

Already done above! Now analyze:

---

### YOUR INTERPRETATION:

**Bullet 1: Top Features**  
[Which features are most important? Why might this be?]

**Bullet 2: Differences**  
[How do built-in vs permutation importance differ?]

**Bullet 3: Business Insight**  
[What does this tell you about the prediction task?]

---

## 7. Comprehensive Model Comparison

In [ ]:
# Compare all models
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

models = {
    'Logistic Regression': Pipeline([
        ('scaler', StandardScaler()),
        ('clf', LogisticRegression(random_state=RANDOM_SEED, max_iter=1000))
    ]),
    'Decision Tree (tuned)': DecisionTreeClassifier(max_depth=5, random_state=RANDOM_SEED),
    'Random Forest (100)': RandomForestClassifier(n_estimators=100, random_state=RANDOM_SEED, n_jobs=-1),
    'Random Forest (200)': RandomForestClassifier(n_estimators=200, random_state=RANDOM_SEED, n_jobs=-1)
}

comparison_results = []
for name, model in models.items():
    cv_results = cross_val_score(model, X_train, y_train, cv=cv, scoring='roc_auc')
    
    # Fit and test
    model.fit(X_train, y_train)
    if hasattr(model, 'predict_proba'):
        test_score = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
    else:
        test_score = roc_auc_score(y_test, model.predict(X_test))
    
    comparison_results.append({
        'Model': name,
        'CV_Mean': cv_results.mean(),
        'CV_Std': cv_results.std(),
        'Test_Score': test_score
    })

final_comparison = pd.DataFrame(comparison_results).sort_values('CV_Mean', ascending=False)
print("=== FINAL MODEL COMPARISON ===")
print(final_comparison.to_string(index=False))

# Visualize
fig, ax = plt.subplots(figsize=(10, 6))
x = np.arange(len(final_comparison))
ax.bar(x - 0.2, final_comparison['CV_Mean'], 0.4, label='CV Mean', alpha=0.8)
ax.bar(x + 0.2, final_comparison['Test_Score'], 0.4, label='Test', alpha=0.8)
ax.set_xlabel('Model')
ax.set_ylabel('ROC-AUC')
ax.set_title('Model Comparison: CV vs Test Performance')
ax.set_xticks(x)
ax.set_xticklabels(final_comparison['Model'], rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

best_model = final_comparison.iloc[0]['Model']
print(f"\n✓ Champion model: {best_model}")

## 8. Wrap-Up: Key Takeaways

### What We Learned Today:

1. **Bagging Reduces Variance**: Averaging many trees stabilizes predictions
2. **Random Forests**: Bagging + random feature selection = powerful ensemble
3. **Hyperparameter Tuning**: n_estimators and max_features most important
4. **OOB Score**: Free cross-validation estimate
5. **Permutation Importance**: More reliable than built-in Gini importance

### Critical Rules:

> **"More trees is almost always better (diminishing returns after 100-500)"**

> **"Use permutation importance for production, not built-in importance"**

> **"Random forests rarely overfit badly (but can underfit)"**

### Next Steps:

- Day 13: Gradient Boosting (sequential ensembles)
- Boosting will give even better performance
- But requires more careful tuning

---

## Bibliography

- Breiman, L. (2001). "Random Forests." *Machine Learning*, 45(1), 5-32.
- James, G., Witten, D., Hastie, T., & Tibshirani, R. (2021). *An Introduction to Statistical Learning with Python* - Tree-Based Methods (bagging/forests)
- Hastie, T., Tibshirani, R., & Friedman, J. (2009). *The Elements of Statistical Learning* - Random forests and bagging
- scikit-learn User Guide: [RandomForest estimators](https://scikit-learn.org/stable/modules/ensemble.html#forest)
- scikit-learn User Guide: [Permutation importance](https://scikit-learn.org/stable/modules/permutation_importance.html)

---

**End of Day 12 Notebook**